In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [107]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetTemizle(DataSet)
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
                  
    def Ceviri(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
                DataSet["Tweet_Ceviri"][yazi]=DataSet["Tweet_Ceviri"][yazi].lower()
            print ("Tweetler Çevirildi!")                        
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            DataSet["Tür"] = getTweets.getPolarity(DataSet["Tweet_Ceviri"])
            DataSet["Yoğunluk"] = getTweets.getSubjectivity(DataSet["Tweet_Ceviri"])
            
    def TweetTemizle(Dataset):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
            for i in range(len(Dataset["Tweet_Degistirilmis"])):        
                Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])    
                Dataset["Tweet_Degistirilmis"][i]=Dataset["Tweet_Degistirilmis"][i].lower()                
            Dataset["Tweet_Degistirilmis"]=Dataset["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")            
            return Dataset
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.Ceviri(Dataset)
    
    def KelimeAyristirma(DataSet):
        try:
            data_test = DataSet.copy()
            stop = stopwords.words('english')
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])
        except Exception as err:
            print('Error: {}'.format(err))
    
    def turkceAnaliz(self,Dataset):
        return 
    
    def getSubjectivity(self,Dataset):
        self.setAuth()
        try:        
            return TextBlob(Dataset).sentiment.subjectivity
        except Exception as err:
            print('Error: {}'.format(err))
    
    def getPolarity(self,Dataset):
        self.setAuth()
        try:
            return TextBlob(Dataset).sentiment.polarity
        except Exception as err:
            print('Error: {}'.format(err))

In [108]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Error: getPolarity() missing 1 required positional argument: 'Dataset'


In [112]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Tür,Yoğunluk
0,1611462629205090324,bugün de cem adrian tarafından bıçaklandığıma göre uyuyabilirim iyi geceler,bugün de cem adrian tarafından bıçaklandığıma göre uyuyabilirim iyi geceler,"since i was stabbed by cem adrian today, i can sleep, good night",2023-01-06 20:40:04+00:00,1275023367456858113,b e t ü l,tr,NaN,NaN
1,1611462623119155214,RT @yinemibekizim: ben ve her yaptıgımı destekleyen iki arkadasım https://t.co/MkkmXghNnq,rt ben ve her yaptıgımı destekleyen iki arkadasım,rt me and my two friends who support everything i do,2023-01-06 20:40:03+00:00,2740028699,yaşamdan yazılar,tr,NaN,NaN
2,1611462620749373446,Konya İl Başkanımız Remzi Karaarslan beyi Konya Rektörümüz Cem Zorlu beyi İl Emniyet Müdürümüz Engin Dinç beyi Mak… https://t.co/YYRvkvh8EI,konya il başkanımız remzi karaarslan beyi konya rektörümüz cem zorlu beyi il emniyet müdürümüz engin dinç beyi mak,konya provincial chairman remzi karaarslan bey konya rector cem zorlu bey provincial police chief engin dinç bey mak mak,2023-01-06 20:40:02+00:00,1227539710400192512,Bilal Yiğit,tr,NaN,NaN
3,1611462614625705984,"RT @ZAMajans: Gökhan Taşkın, Temel Karamollaoğlu'nun ""İşi uzatmayalım Kemal Kılıçdaroğlu adayımız olsun."" dediğini aktardı. https://t.co/1S…",rt gökhan taşkın temel karamollaoğlu nun işi uzatmayalım kemal kılıçdaroğlu adayımız olsun dediğini aktardı,rt gökhan taşkın temel karamollaoğlu'nun let us not extend the work kemal kılıçdaroğlu said he said get our candidate,2023-01-06 20:40:01+00:00,1018173972909907969,Cem Savan,tr,NaN,NaN
4,1611462539191238656,https://t.co/K807qjzKod Körfez İzmit YT Karamürsel #İzmit Cem Neyim Kartepe özetlemiş Şans Bana,körfez izmit yt karamürsel izmit cem neyim kartepe özetlemiş şans bana,gulf izmit yt karamürsel izmit cem neyim kartepe summarized luck to me,2023-01-06 20:39:43+00:00,1595155786249998337,Ahunaz êſčòrțſ,tr,NaN,NaN


In [113]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Tür,Yoğunluk
5,1611462537320472586,"RT @Daricabeltr: Belediye Başkanvekilimiz Göknur Bektaş, Darıca Anadolu Kültür Derneği ve Cem Evi tarafından düzenlenen halk konserine katı…",rt belediye başkanvekilimiz göknur bektaş darıca anadolu kültür derneği ve cem evi tarafından düzenlenen halk konserine katı,rt mayor göknur bektaş darıca anatolian culture association and cem evi organized by the public concert,2023-01-06 20:39:43+00:00,1261076850044022791,TispePremius,tr,NaN,NaN
6,1611462427165466642,Boğa gelene kadar buralar saçmalamaya devam edecek. Türkiye trade ile para kazanmaya çalışanların ülkesi değil. Kri… https://t.co/YbIFMvlj0Q,boğa gelene kadar buralar saçmalamaya devam edecek türkiye trade ile para kazanmaya çalışanların ülkesi değil kri,"until the taurus comes, turkey will continue to be nonsense with turkey, but the country of those who try to earn money kri",2023-01-06 20:39:16+00:00,1372994944709881866,Kripto Cem,tr,NaN,NaN
7,1611462413660073984,@esin_oran Cem küçük,cem küçük,cem küçük,2023-01-06 20:39:13+00:00,1425858657477177346,Hatice Özdemir,tr,NaN,NaN
8,1611462299960725505,RT @eythayal_degil: @bilgen91828960 @RTErdogan @dbdevletbahceli @Mustafa_Destici @fuatoktay @MustafaSentop @vedatbilgn @NureddinNebati @AvO…,rt,rt,2023-01-06 20:38:46+00:00,1545430982123294721,abdullah kmrc,tr,NaN,NaN
9,1611462270235688960,RT @eythayal_degil: @Murat11541752 @AbdurrahimFIRAT @ab_kurt @ahmetgundogdu01 @AhmetSCeylan @alaattinparlak @aliihsanyavuz54 @ayhan_salman…,rt,rt,2023-01-06 20:38:39+00:00,1592868652533092352,Nilüfer Duran,tr,NaN,NaN


In [114]:
savedTweets = Veri
savedTweets['Yoğunluk'] = savedTweets['Tweet_Ceviri'].apply(tw.getSubjectivity)
savedTweets['Tür'] = savedTweets['Tweet_Ceviri'].apply(tw.getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!
Bağlantı Yapıldı!


,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Tür,Yoğunluk,Subjectivity,Polarity,Duygu Turkce
0,1611462629205090324,bugün de cem adrian tarafından bıçaklandığıma göre uyuyabilirim iyi geceler,bugün de cem adrian tarafından bıçaklandığıma göre uyuyabilirim iyi geceler,"since i was stabbed by cem adrian today, i can sleep, good night",2023-01-06 20:40:04+00:00,1275023367456858113,b e t ü l,tr,NaN,NaN,0.6,0.7,None
1,1611462623119155214,RT @yinemibekizim: ben ve her yaptıgımı destekleyen iki arkadasım https://t.co/MkkmXghNnq,rt ben ve her yaptıgımı destekleyen iki arkadasım,rt me and my two friends who support everything i do,2023-01-06 20:40:03+00:00,2740028699,yaşamdan yazılar,tr,NaN,NaN,0.0,0.0,None
2,1611462620749373446,Konya İl Başkanımız Remzi Karaarslan beyi Konya Rektörümüz Cem Zorlu beyi İl Emniyet Müdürümüz Engin Dinç beyi Mak… https://t.co/YYRvkvh8EI,konya il başkanımız remzi karaarslan beyi konya rektörümüz cem zorlu beyi il emniyet müdürümüz engin dinç beyi mak,konya provincial chairman remzi karaarslan bey konya rector cem zorlu bey provincial police chief engin dinç bey mak mak,2023-01-06 20:40:02+00:00,1227539710400192512,Bilal Yiğit,tr,NaN,NaN,0.0,0.0,None
3,1611462614625705984,"RT @ZAMajans: Gökhan Taşkın, Temel Karamollaoğlu'nun ""İşi uzatmayalım Kemal Kılıçdaroğlu adayımız olsun."" dediğini aktardı. https://t.co/1S…",rt gökhan taşkın temel karamollaoğlu nun işi uzatmayalım kemal kılıçdaroğlu adayımız olsun dediğini aktardı,rt gökhan taşkın temel karamollaoğlu'nun let us not extend the work kemal kılıçdaroğlu said he said get our candidate,2023-01-06 20:40:01+00:00,1018173972909907969,Cem Savan,tr,NaN,NaN,0.0,0.0,None
4,1611462539191238656,https://t.co/K807qjzKod Körfez İzmit YT Karamürsel #İzmit Cem Neyim Kartepe özetlemiş Şans Bana,körfez izmit yt karamürsel izmit cem neyim kartepe özetlemiş şans bana,gulf izmit yt karamürsel izmit cem neyim kartepe summarized luck to me,2023-01-06 20:39:43+00:00,1595155786249998337,Ahunaz êſčòrțſ,tr,NaN,NaN,0.0,0.0,None
